In [1]:
%%capture
# Required installations
!pip install grpcio==1.63.0
!pip install crewai==0.41.1
!pip install 'crewai[tools]'
!pip install langchain-google-genai==1.0.4
!pip install langchain-anthropic==0.1.20
!pip install langchain_milvus==0.1.3
!pip install langchain-openai==0.1.19
!pip install langchain-community==0.2.10
!pip install sentence-transformers==3.0.1
!pip install protobuf==5.27.2
!npm install -g degit


In [2]:
from crewai import Agent, Task, Crew
from google.colab import userdata
import os

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
model_name = "gpt-4o-mini"

open_ai_llm = ChatOpenAI(
    model = model_name,
    temperature = 0.5)

## Nhân viên thứ nhất

In [4]:
class_planner = Agent(
    role="Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy",
    goal="Lập kế hoạch các chủ đề hàng tuần hấp dẫn và giáo dục",
    backstory="Bạn chịu trách nhiệm lập kế hoạch các chủ đề hàng tuần.  "
              "Bạn cần đảm bảo nội dung mang tính giáo dục, hấp dẫn và phù hợp với tiêu chuẩn chương trình học. "
              "Kế hoạch của bạn sẽ giúp giáo viên cung cấp các bài học toàn diện và thú vị bao gồm "
              "nhiều chủ đề vật lý khác nhau. Mỗi tuần nên xây dựng dựa trên tuần trước, củng cố và mở rộng kiến thức vật lý của học sinh, giúp học sinh được rèn năng lực vận dụng vật lý"
              "Kĩ thuật dạy học: Kĩ thuật chia nhóm, kĩ thuật đặt câu hỏi, hỏi đáp, động não",
    llm=open_ai_llm,
    allow_delegation=False,
    verbose=True
)
planning_task = Task(
    description=(
        "1. Xác định các chủ đề quan trọng và phù hợp cho {topic} trong vòng {range} và phù hợp với các học sinh {students} "
        "bao gồm các khái niệm cơ bản, định lý, và có thể bao gồm ứng dụng thực tế của {sub_topics}\n"
        "2. Xác định đối tượng học sinh, xem xét "
        "sở thích và những khó khăn của họ trong việc học môn Vật Lý.\n"
        "3. Phát triển một dàn ý chi tiết cho mỗi tuần bao gồm "
        "phần giới thiệu, kỹ thuật dạy học, các điểm chính, hoạt động học tập và câu hỏi thảo luận.\n"
        "4. Bao gồm các từ khóa quan trọng và các nguồn tài liệu tham khảo liên quan."
        "5. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một tài liệu kế hoạch nội dung toàn diện "
        "với dàn ý hàng tuần theo tiết học cụ thể, phân tích đối tượng học sinh, "
        "từ khóa quan trọng và các nguồn tài liệu tham khảo.",
    output_file="planning.md",
    agent=class_planner,
)


## Nhân viên thứ hai

In [6]:
lesson_designer = Agent(
    role="Giáo viên Thiết Kế Bài Học Vật Lý",
    goal="Thiết kế các bài học chi tiết và chính xác về mặt thông tin cho môn Vật Lý lớp 10, mỗi tiết học dài 45 phút",
    backstory="Bạn đang làm việc để thiết kế các bài học mới cho môn Vật Lý 10. "
              "Bạn dựa vào công việc của Giáo viên Vật Lý lớp 10 Lập Kế Hoạch Nội Dung Giảng dạy, người cung cấp dàn ý "
              "và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tuân theo các mục tiêu chính và "
              "hướng dẫn của dàn ý, "
              "do Giáo viên Vật Lý lớp 10 Lập Kế Hoạch Nội Dung Giảng dạy cung cấp. "
              "Bạn cũng cung cấp các thông tin khách quan và không thiên vị "
              "và củng cố chúng bằng các thông tin "
              "được cung cấp bởi Giáo viên Vật Lý lớp 10 Lập Kế Hoạch Nội Dung Giảng dạy. "
              "Bạn đảm bảo bài học của mình có tính tương tác, hấp dẫn và phù hợp với tiêu chuẩn chương trình học.",
    allow_delegation=False,
    llm=open_ai_llm,
    verbose=True
)

lesson_plan_task = Task(
    description=(
        "1. Xác định mục tiêu học tập cho tiết học 45 phút về chủ đề {topic}.\n"
        "2. Lập dàn ý chi tiết cho tiết học, bao gồm phần giới thiệu, các điểm chính, hoạt động học tập, và kết luận.\n"
        "3. Phát triển các hoạt động học tập tương tác và hấp dẫn để giúp học sinh hiểu sâu hơn về chủ đề.\n"
        "4. Bao gồm các câu hỏi thảo luận và bài tập để kiểm tra sự hiểu biết của học sinh.\n"
        "5. Đảm bảo nội dung phù hợp với tiêu chuẩn chương trình học và phù hợp với trình độ của học sinh.\n"
        "6. Cung cấp các nguồn tài liệu tham khảo liên quan để hỗ trợ bài học. "
        "7. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một kế hoạch bài học chi tiết cho tiết học 45 phút về chủ đề {topic}, "
        "bao gồm mục tiêu học tập, dàn ý chi tiết, hoạt động học tập, câu hỏi thảo luận, bài tập, và các nguồn tài liệu tham khảo.",
    output_file="baigiang.md",
    agent=lesson_designer,
)


In [7]:
crew = Crew(
    agents=[class_planner, lesson_designer],
    tasks=[planning_task,lesson_plan_task],
    verbose=2
)

In [8]:
result = crew.kickoff(inputs={'topic': 'Môn vật lý lớp 10',
                              'range': '6 tuần',
                              'sub_topics': 'động học, độ dịch chuyển',
                              "students": "30 em học sinh giỏi, 15 em học sinh khá, và 5 em học sinh trung bình"})
print(result)

 [2024-08-10 13:36:43][DEBUG]: == Working Agent: Giáo viên Lập Kế Hoạch Nội Dung Giảng dạy
 [2024-08-10 13:36:43][INFO]: == Starting Task: 1. Xác định các chủ đề quan trọng và phù hợp cho Môn vật lý lớp 10 trong vòng 6 tuần và phù hợp với các học sinh 30 em học sinh giỏi, 15 em học sinh khá, và 5 em học sinh trung bình bao gồm các khái niệm cơ bản, định lý, và có thể bao gồm ứng dụng thực tế của động học, độ dịch chuyển
2. Xác định đối tượng học sinh, xem xét sở thích và những khó khăn của họ trong việc học môn Vật Lý.
3. Phát triển một dàn ý chi tiết cho mỗi tuần bao gồm phần giới thiệu, kỹ thuật dạy học, các điểm chính, hoạt động học tập và câu hỏi thảo luận.
4. Bao gồm các từ khóa quan trọng và các nguồn tài liệu tham khảo liên quan.5. Tất cả các output bằng tiếng Việt.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer: 

**Kế hoạch giảng dạy môn Vật Lý lớp 10 trong 6 tuần**

### Tuần 1: Giới thiệu về Động học
- **Giới thiệu**: Động họ

In [9]:
print("-----------------------------------------END OF 2 AGENTS---------------------------------------")

-----------------------------------------END OF 2 AGENTS---------------------------------------


## Nhân Viên Thứ 3

In [4]:
!npx degit STEAMforVietnam/genai-agents-for-teachers/tools tools --force

> cloned STEAMforVietnam/genai-agents-for-teachers#HEAD to tools


In [5]:
os.environ["DATABSE_PUBLIC_ENDPOINT"] = userdata.get("DATABSE_PUBLIC_ENDPOINT")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [6]:
from tools.exam import ExamTool

In [7]:
test_creator = Agent(
    role="Giáo viên tạo Bài Kiểm Tra và Bài Tập Vật Lý",
    goal="Tạo các bài kiểm tra và bài tập chính xác và phù hợp cho từng chủ đề của môn Vật Lý 10 ",
    backstory="Bạn chịu trách nhiệm tạo các bài kiểm tra và bài tập cho môn Vật Lý 10. "
              "Bạn dựa vào công việc của Giáo viên Lập Kế Hoạch Nội Dung và Giáo viên Thiết Kế Bài Học, "
              "những người cung cấp dàn ý và ngữ cảnh liên quan đến chủ đề. "
              "Bạn tạo các bài kiểm tra và bài tập nhằm đánh giá sự hiểu biết của học sinh "
              "và củng cố kiến thức đã học. "
              "Bạn đảm bảo các bài kiểm tra và bài tập phù hợp với tiêu chuẩn chương trình học "
              "và trình độ của học sinh.",
    allow_delegation=False,
    llm=open_ai_llm,
    verbose=True,
    tools=[ExamTool.get_chapter]
)


test_assignment_task = Task(
    description=(
        "1. Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề {sub_topics}\n"
        "2. Tạo các câu hỏi trắc nghiệm và tự luận để đánh giá sự hiểu biết của học sinh về từng chủ đề trong {sub_topics}.\n"
        "3. Phát triển các bài tập để học sinh thực hành và củng cố kiến thức.\n"
        "4. Bao gồm các câu hỏi và bài tập ở nhiều mức độ khó khác nhau để phù hợp với trình độ của học sinh, đi từ dễ đến khó, bao gồm các bài kiểm tra 15 phút và một tiết (45 phút).\n"
        "5. Đảm bảo các câu hỏi và bài tập phù hợp với tiêu chuẩn chương trình học.\n"
        "6. Cung cấp hướng dẫn chấm điểm và đáp án cho các câu hỏi và bài tập."
        "7. Tất cả các output bằng tiếng Việt."
    ),
    expected_output="Một bộ đề kiểm tra và bài tập chi tiết cho chủ đề {topic} và theo từng {sub_topics}"
        "bao gồm câu hỏi trắc nghiệm, câu hỏi tự luận, bài tập, hướng dẫn chấm điểm và đáp án. Trả lại output theo dạng Markdown",
    agent=test_creator,
)

In [8]:
crew = Crew(
    # agents=[class_planner, lesson_designer, test_creator],
    agents=[test_creator],
    # tasks=[planning_task,lesson_plan_task, test_assignment_task],
    tasks=[test_assignment_task],
    verbose=2
)


In [9]:
result = crew.kickoff(inputs={'topic': 'Môn vật lý lớp 10',
                              'sub_topics': 'động học, độ dịch chuyển',
                              })
print(result)

 [2024-08-10 13:55:43][DEBUG]: == Working Agent: Giáo viên tạo Bài Kiểm Tra và Bài Tập Vật Lý
 [2024-08-10 13:55:43][INFO]: == Starting Task: 1. Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề động học, độ dịch chuyển
2. Tạo các câu hỏi trắc nghiệm và tự luận để đánh giá sự hiểu biết của học sinh về từng chủ đề trong động học, độ dịch chuyển.
3. Phát triển các bài tập để học sinh thực hành và củng cố kiến thức.
4. Bao gồm các câu hỏi và bài tập ở nhiều mức độ khó khác nhau để phù hợp với trình độ của học sinh, đi từ dễ đến khó, bao gồm các bài kiểm tra 15 phút và một tiết (45 phút).
5. Đảm bảo các câu hỏi và bài tập phù hợp với tiêu chuẩn chương trình học.
6. Cung cấp hướng dẫn chấm điểm và đáp án cho các câu hỏi và bài tập.7. Tất cả các output bằng tiếng Việt.


> Entering new CrewAgentExecutor chain...
Action: Get Chapter Tool  
Action Input: {"query":"động học, độ dịch chuyển","subject":"vat_ly"}  

KeyboardInterrupt: 

In [ ]:
result.raw

'### Kế Hoạch Bài Học Môn Vật Lý Lớp 9: Động Học\n\n#### 1. Mục Tiêu Học Tập\n- Học sinh hiểu được khái niệm động học và các đại lượng chuyển động như vị trí, độ dịch chuyển, và vận tốc.\n- Học sinh có khả năng tính toán vận tốc trung bình và phân biệt giữa vị trí và độ dịch chuyển.\n- Học sinh áp dụng kiến thức vào các tình huống thực tế trong cuộc sống hàng ngày.\n\n#### 2. Dàn Ý Chi Tiết\n\n**Giới thiệu (5 phút):**\n- Giới thiệu khái niệm động học và tầm quan trọng của nó trong vật lý.\n- Đặt câu hỏi khởi động: "Khi nào các bạn nghĩ về chuyển động trong cuộc sống hàng ngày?"\n\n**Các Điểm Chính (20 phút):**\n- **Khái niệm Vị Trí và Độ Dịch Chuyển:**\n  - Định nghĩa vị trí: Điểm trong không gian mà một đối tượng chiếm giữ.\n  - Định nghĩa độ dịch chuyển: Khoảng cách giữa vị trí ban đầu và vị trí cuối cùng.\n- **Công Thức Tính Vận Tốc:**\n  - Vận tốc = Độ dịch chuyển / Thời gian.\n  - Ví dụ minh họa: Tính vận tốc của một chiếc xe từ nhà đến trường.\n\n**Hoạt Động Học Tập (15 phút):**\

In [ ]:
with open("./file.md", "w") as file:
  file.write(result.raw)